In [8]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np
import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [14]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [2]:
df = pd.read_excel('data/gb_sem_8_hm.xlsx')

C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [4]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [5]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [6]:
df_new.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [12]:
control = df[df['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = df[df['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [11]:
fig = px.histogram(df,
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='revenue_distribution',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()

In [15]:
continious_result(control, treatment, column='REVENUE')

100%|██████████| 10000/10000 [00:06<00:00, 1495.84it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.028393,0.207544,0.757808,"[-0.023, 0.207]",0.074169


#### Рекомендации менеджеру: Необходимо доработать систему сплитования, поскольку более 30% данных пришлось ликвидировать из-за некорректного сплитования юзеров - они часто попадали в обе группы тестирования. Сам тест показал, что статистическую разницу между результатами в двух группах нельзя доказать, поскольку 0 входит в доверительный интервал, значения альфа и бета слишком велики. Соответственно, изменения нельзя принять. ####